The purpose of this notebook is to identify the frequency of conflicts between metrics that purport to measure the degree to which a plan favors one party or another.

In [1]:
from typing import List, Dict, Any, Set

import os
import pandas as pd
from collections import defaultdict

from rdapy import DISTRICTS_BY_STATE
from rdametrics import states, chambers, ensembles

Load the scores dataframe

In [2]:
scores_path: str = "~/local/beta-ensembles/dataframe/contents/scores_df.parquet"
scores_df = pd.read_parquet(os.path.expanduser(scores_path))

Helper code

In [3]:
ensembles = [e for e in ensembles if e not in ["A1", "A2", "A3", "A4", "Rev*"]]

partisan_metrics: List[str] = [
    "disproportionality",
    "efficiency_gap",
    "seats_bias",
    "votes_bias",
    "geometric_seats_bias",
    "declination",
    "mean_median_statewide",
    "lopsided_outcomes",
]


def same_sign(a, b):
    if a == 0.0 or b == 0.0:
        return True

    return a * b > 0

Count instances where the scores conflict

In [4]:
counters: Dict[str, Dict[str, Any]] = dict()
for m in partisan_metrics[1:]:
    counters[m] = {
        "combinations": set(),
        "conflicts": 0,
        "example": None,
        "value": None,
        "disproportionality": None,
        "delta": None,
    }

total_plans: int = len(scores_df)
for index, row in scores_df.iterrows():
    xx: str = row["state"]
    chamber: str = row["chamber"]
    ensemble: str = row["ensemble"]

    if ensemble not in ensembles:
        continue

    for m in partisan_metrics[1:]:
        if same_sign(row[m], row["disproportionality"]):
            continue
        else:
            counters[m]["combinations"].add((xx, chamber, ensemble))
            counters[m]["conflicts"] += 1

            delta: float = abs(row[m] - row["disproportionality"])

            if counters[m]["example"] is None or delta > counters[m]["delta"]:
                counters[m]["example"] = row["map"]
                counters[m]["value"] = row[m]
                counters[m]["disproportionality"] = row["disproportionality"]
                counters[m]["delta"] = delta

Report the results

In [5]:
print("Partisan Conflicts Summary:")
for m in partisan_metrics[1:]:
    conflicts: int = counters[m]["conflicts"]
    combos: int = len(counters[m]["combinations"])
    name: str = counters[m]["example"]
    sample: float = counters[m]["value"]
    disp: float = counters[m]["disproportionality"]
    print(
        f"- {m}: {conflicts:,} of {total_plans:,} conflicts ({conflicts / total_plans:.1%})"
    )
    print(
        f"  across {combos} of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations."
    )
    print(f"  Example: Map {name} has {sample:.4f} vs. disproportionality {disp:.4f}.")
    print()
print()
print(
    f"Where a 'conflict' is when the sign of the metric is the *opposite* of the sign for simple 'disproportionality'."
)

Partisan Conflicts Summary:
- efficiency_gap: 680,747 of 6,719,956 conflicts (10.1%)
  across 165 of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations.
  Example: Map 7460000 has 0.0050 vs. disproportionality -0.1428.

- seats_bias: 1,510,956 of 6,719,956 conflicts (22.5%)
  across 176 of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations.
  Example: Map 29372500 has 0.0930 vs. disproportionality -0.2419.

- votes_bias: 1,509,410 of 6,719,956 conflicts (22.5%)
  across 176 of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations.
  Example: Map 40532500 has 0.0389 vs. disproportionality -0.2511.

- geometric_seats_bias: 1,545,812 of 6,719,956 conflicts (23.0%)
  across 176 of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations.
  Example: Map 31252500 has 0.0109 vs. disproportionality -0.2466.

- declination: 1,385,292 of 6,719,956 conflicts (20.6%)
  across 165 of 231 = 7 x 3 x 11 state, chamber, and ensemble combinations.
  Example: Map 48322500 has 25.94